**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Flatten, Conv2D, MaxPooling2D, Activation, AveragePooling2D, Reshape, BatchNormalization

C:\Users\Driss Debbagh\Anaconda3\lib\site-packages\h5py\__init__.py:72: UserWarning: h5py is running against HDF5 1.10.2 when it was built against 1.10.3, this may cause problems
  '{0}.{1}.{2}'.format(*version.hdf5_built_version_tuple)
Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self, e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

> The function **act** return which action to make for the next step. The actions are integers from 0 to 'n_action'. The action is chosen in two different ways: randomly or using the learned actions. We have **epsilon %** of chosing a random action and **1 - epsilon %** chosing an action based on the learned one.

> Epsilon is essential because its value can make big differences on the actions that are made. Indeed a an epsilon value of 0 implies actions that are only learned, unlike when its value is 1 and all the actions are made randomly.
Here is an example to explain why it's important to consider a value >0 of epsilon:
Imagine a situation with two possible actions and two possible states:
- action 1 with probability 1 and reward of 1
- action 2 with probability 1/2 and reward of 3

> Let's consider now that we took the value 0 for epsilon. 
If in the first time we choose action 2, we obtain a reward of 0 then after that we will always choose action 1. The mean reward will be of 1 instead of 1.5 if we have always chosen action 2.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
                
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [55]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=51 # set small when debugging
epochs_test=51 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

- Array position:
> This matrix contains the position of the rat and the positions where he can or can not go (respectively 1, 0, -1). 

- Array board
> This matrix contains the rewards on each position (-1 poison, 0.5 cheese, 0 nothing). And whenever the rat goes to a position the board takes the value 0 (meaning that the rat took already the cheese or the poison)

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        random_action = np.random.randint(0, self.n_action, size=1)[0]
        return random_action

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [9]:
def test(agent, env, epochs, prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):

        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        env.draw(prefix + str(e))

        # Update stats
        score = score + win-lose

#         print("Win/lose count {}/{}. Average score ({})"
#               .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))
    return score/epochs
    

In [10]:
# Initialize the game
env = Environment(grid_size=size, max_time=T, temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test, prefix='Random/random')

Win/lose count 13.0/10.0. Average score (3.0)
Win/lose count 16.0/17.0. Average score (1.0)
Win/lose count 11.0/14.0. Average score (-0.3333333333333333)
Win/lose count 7.5/12.0. Average score (-1.375)
Win/lose count 12.0/12.0. Average score (-1.1)
Win/lose count 14.5/23.0. Average score (-2.3333333333333335)
Win/lose count 11.5/25.0. Average score (-3.9285714285714284)
Win/lose count 11.5/13.0. Average score (-3.625)
Win/lose count 10.0/22.0. Average score (-4.555555555555555)
Win/lose count 14.0/12.0. Average score (-3.9)
Win/lose count 6.5/8.0. Average score (-3.6818181818181817)
Final score: -3.6818181818181817


-3.6818181818181817

In [11]:
HTML(display_videos('Random/random50.mp4'))

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




#### Answer

1. \begin{equation*}Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]\end{equation*}

> We have from the defition given above:

\begin{equation*}
\begin{aligned}
Q^\pi(s,a)&=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \>\\
&= E_{p^{\pi}}[r(s_0,a_0) + \sum_{1 \leq t\leq \infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> \\
&= r(s,a) + \gamma E_{p^{\pi}}[ \sum_{1 \leq t\leq \infty}\gamma^{t-1}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> \\
&= r(s,a)+ \gamma E_{s',a'} P(s_1 = s', a_1 = a' | s_0 = s, a_0 = a) E_{p^{\pi}}[\sum_{0 \leq t\leq \infty} \gamma^t r(s_t,a_t)|s_0 = s', a_0 = a']\\
&= r(s,a) + \gamma E_{(s',a')\sim p(.|s,a)}[Q^{\pi}(s',a')]\\
&= E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{aligned}
\end{equation*}



> Finally, $\boxed{Q^{\pi}(s,a) = E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]}$


2. \begin{equation*}Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].\end{equation*}

> We found on the previous question:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')] =E_{s'\sim p(.|s,a)}[r(s,a)+\gamma E_{a'\sim \pi}[Q^{\pi}(s',a')]]
\end{equation*}
Since we have : 

\begin{equation*}Q^*(s,a) = max_{\pi}Q^{\pi}(s,a)\end{equation*}

Then:

\begin{equation*}
\begin{aligned}
Q^*(s,a)
&= max_{\pi}E_{s'\sim p(.|s,a)}[r(s,a)+\gamma E_{a'\sim \pi}[Q^{\pi}(s',a')]]\\
&= max_{\pi}[{r(s,a)+\gamma E_{s'\sim p(.|s,a)}}[E_{a'\sim \pi}[Q^{\pi}(s',a')]]]\\
&= r(s,a)+\gamma max_{\pi}E_{s'\sim p(.|s,a)}[E_{a'\sim \pi}[Q^{\pi}(s',a')]]\\
&=r(s,a)+\gamma E_{s'\sim \pi^*(.|s,a)}[E_{a'\sim \pi*}[Q^{*\pi}(s',a')]]  & (\pi^*(s) = argmax_a Q^*(s,a))\\
&= r(s,a)+\gamma E_{s'\sim \pi^*(.|s,a)}[Q^*(s',\pi^*(s'))] \\
&= E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma max_a'Q^*(s',a')]
\end{aligned}
\end{equation*}


>Finally, we have: $\boxed{Q^*(s,a) = E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma max_a'Q^*(s',a')]}$

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [12]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)
        # Delete first move if max_memory reached
        if len(self.memory) > self.max_memory:
            del self.memory[0]

    def random_access(self):
        random_index = np.random.randint(0, len(self.memory))
        return self.memory[random_index]

***
The pipeline we will use for training is given below:

In [13]:
def train(agent, env, epoch, prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
    print('Final mean score: ' + str(score/epoch))
    agent.save(name_weights=prefix+'model.h5', name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [20]:
class DQN(Agent):
    def __init__(self, grid_size, epsilon=0.1, memory_size=100, batch_size=16, n_state=2):
        super(DQN, self).__init__(epsilon=epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(np.array([s])))

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5, 5, self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            ######## FILL IN
            [s_, n_s_, a_, r_, game_over_] = self.memory.random_access()
            input_states[i] = s_
            target_q[i] = self.model.predict(np.array([s_]))[0]

            if game_over_:
                target_q[i][a_] = r_
            else:
                q_n_s_ = self.model.predict(np.array([n_s_]))
                target_q[i][a_] = r_ + self.discount * max(q_n_s_[0])
                
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)
        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        ####### FILL IN   
        model = Sequential()
        model.add(Flatten(input_shape=(5, 5, self.n_state,)))
        model.add(Dense(32, activation="relu"))
        model.add(Dense(16, activation="relu"))
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        
#         print("### FC model ###")
#         print(model.summary())

In [45]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='FC_train/fc_train')

### FC model ###
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_11 (Flatten)         (None, 50)                0         
_________________________________________________________________
dense_24 (Dense)             (None, 32)                1632      
_________________________________________________________________
dense_25 (Dense)             (None, 16)                528       
_________________________________________________________________
dense_26 (Dense)             (None, 4)                 68        
Total params: 2,228
Trainable params: 2,228
Non-trainable params: 0
_________________________________________________________________
None
Epoch 000/051 | Loss 0.0735 | Win/lose count 5.0/7.0 (-2.0)
Epoch 001/051 | Loss 0.0103 | Win/lose count 3.5/5.0 (-1.5)
Epoch 002/051 | Loss 0.0093 | Win/lose count 3.5/1.0 (2.5)
Epoch 003/051 | Loss 0.0078 | Win/lose count 4.5/2.0 (2.5)
Epoch 004/05

In [46]:
HTML(display_videos('FC_train/fc_train50.mp4'))

> We obtain better results than the random choices tried before. Now we will implement another Network and see if it performs better.

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [21]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        # Convolutional Deep Network 
        model_cnn = Sequential()
        
        # Convolutional Layers
        model_cnn.add(Conv2D(16, (2, 2), activation='relu', input_shape=(5, 5, self.n_state,)))
        model_cnn.add(Conv2D(32, (2, 2), activation='relu'))
        
        #Flattening before Fully Connected Layers
        model_cnn.add(Flatten())
        model_cnn.add(Dense(40, activation='relu'))
        model_cnn.add(Dense(4))

        model_cnn.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model_cnn
        
#         print("### CNN model ###")
#         print(model_cnn.summary())

In [43]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='CNN_train/cnn_train')

### CNN model ###
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_21 (Conv2D)           (None, 4, 4, 16)          144       
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 3, 3, 32)          2080      
_________________________________________________________________
flatten_10 (Flatten)         (None, 288)               0         
_________________________________________________________________
dense_22 (Dense)             (None, 40)                11560     
_________________________________________________________________
dense_23 (Dense)             (None, 4)                 164       
Total params: 13,948
Trainable params: 13,948
Non-trainable params: 0
_________________________________________________________________
None
Epoch 000/051 | Loss 0.0016 | Win/lose count 3.0/7.0 (-4.0)
Epoch 001/051 | Loss 0.0030 | Win/lose count 5.5/5.0 (0

In [44]:
HTML(display_videos('CNN_train/cnn_train50.mp4'))

> We obtained the best result so far using a Convolution Neural Network with 2 Convolution layers with filters that have the same size then a 2 Fully connected layers.

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [55]:
# Common environment 
env = Environment(grid_size=size, max_time=T,temperature=0.3)

In [56]:
# Fully Connected Model
agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='FC_train/fc_trainmodel.h5',name_model='FC_train/fc_trainmodel.json')

print('\n---------- Test of the Fully Connected Network ----------')
test(agent_fc, env, epochs_test, prefix='FC_test/fc_test')

### FC model ###
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_14 (Flatten)         (None, 50)                0         
_________________________________________________________________
dense_32 (Dense)             (None, 32)                1632      
_________________________________________________________________
dense_33 (Dense)             (None, 16)                528       
_________________________________________________________________
dense_34 (Dense)             (None, 4)                 68        
Total params: 2,228
Trainable params: 2,228
Non-trainable params: 0
_________________________________________________________________
None

---------- Test of the FC ----------
Win/lose count 5.5/7.0. Average score (-1.5)
Win/lose count 3.5/3.0. Average score (-0.5)
Win/lose count 4.0/7.0. Average score (-1.3333333333333333)
Win/lose count 3.5/3.0. Average score (-0.875)
Win/lose count 

In [57]:
HTML(display_videos('FC_test/fc_test10.mp4'))

In [58]:
# Convolutional Neural Network
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='CNN_train/cnn_trainmodel.h5', name_model='CNN_train/cnn_trainmodel.json')

print('\n---------- Test of the Convolutional Neural Network ----------')
test(agent_cnn, env, epochs_test, prefix='CNN_test/cnn_test')

### CNN model ###
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_25 (Conv2D)           (None, 4, 4, 16)          144       
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 3, 3, 32)          2080      
_________________________________________________________________
flatten_15 (Flatten)         (None, 288)               0         
_________________________________________________________________
dense_35 (Dense)             (None, 40)                11560     
_________________________________________________________________
dense_36 (Dense)             (None, 4)                 164       
Total params: 13,948
Trainable params: 13,948
Non-trainable params: 0
_________________________________________________________________
None

---------- Test of the Convolutional Neural Network ----------
Win/lose count 3.5/2.0. Average score (1.5)
Win/los

In [59]:
HTML(display_videos('CNN_test/cnn_test10.mp4'))

In [74]:
# Variation of Temperature
temperatures = np.linspace(0.1, 0.9, num=9)
scores_fc = []
socres_cnn = []
for temp in temperatures:
    print("Temperature: ", round(temp, 1))
    env = Environment(grid_size=size, max_time=T,temperature=temp)
    # Fully Connected Network
    print("FC Network -", end=' ')
    agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
    agent_cnn.load(name_weights='FC_train/fc_trainmodel.h5',name_model='FC_train/fc_trainmodel.json')
    scores_fc.append(test(agent_fc, env, epochs_test, prefix='FC_test/fc_test'))
    # Convolutional Neural Network
    print("CN Network -", end=' ')
    agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
    agent_cnn.load(name_weights='CNN_train/cnn_trainmodel.h5', name_model='CNN_train/cnn_trainmodel.json')
    socres_cnn.append(test(agent_cnn, env, epochs_test, prefix='CNN_test/cnn_test'))

Temperature:  0.1
FC Network - Final score: -1.5
CN Network - Final score: 2.6363636363636362
Temperature:  0.2
FC Network - Final score: -2.4545454545454546
CN Network - Final score: 5.590909090909091
Temperature:  0.3
FC Network - Final score: -0.18181818181818182
CN Network - Final score: 8.272727272727273
Temperature:  0.4
FC Network - Final score: 0.9545454545454546
CN Network - Final score: 13.727272727272727
Temperature:  0.5
FC Network - Final score: 2.772727272727273
CN Network - Final score: 22.136363636363637
Temperature:  0.6
FC Network - Final score: 1.1363636363636365
CN Network - Final score: 27.09090909090909
Temperature:  0.7
FC Network - Final score: 3.6363636363636362
CN Network - Final score: 27.772727272727273
Temperature:  0.8
FC Network - Final score: 7.136363636363637
CN Network - Final score: 41.45454545454545
Temperature:  0.9
FC Network - Final score: 15.090909090909092
CN Network - Final score: 39.90909090909091


#### Observations

> As we can see, the Convolutional Neural Network performs way better than the Fully Connected one. But they both perform better than the Random model.
The issue observed is that the algorithm does stay in the same area where he started and never cover many areas in the island. The rat never ventures in the island far from where he started looking. 
Even if the rat doesn't get the poison, he also doesn't get the cheese thus the final result is low comapred to what the rat can actually eat (as cheese). We would like the avoid the most possible going back to cells where we already have been.

> There are two different ways to lower the effects of the issue faced:
- Decrease the value of epsilon progressively. It will lower the random part of our algorithm
- Take in account the fact that we already have been into a cell to avoid the less possible going back to it

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [56]:
def train_explore(agent, env, epoch, eps_end=0.1, eps_games=10, prefix=''):
    # Number of won games
    score = 0
    loss = 0
    
    eps_start = agent.epsilon
    
    # Values of epsilon
    list_epsilon = np.linspace(0, eps_start - eps_end, epoch)
    
    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        
        # Setting new epsilon
        agent.set_epsilon(round(eps_start - list_epsilon[e], 2))
        
        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)
            
            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({}) | Epsilon {}"
              .format(e, epoch, loss, win, lose, win-lose, agent.epsilon))
    print('Final mean score: ' + str(score/epoch))
    agent.save(name_weights=prefix+'model.h5', name_model=prefix+'model.json')

In [57]:
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1, param_malus=-0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature
        self.param_malus = param_malus

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train=True):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
                
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')
        
        self.t = self.t + 1
        
        # Initialize the reward
        reward = 0
        reward = self.board[self.x, self.y]
#         print("2. ", reward)
        # Get the information that we've already been in the cell
        if train:
            reward += self.malus[self.x, self.y]
#       print("3.", self.malus[self.x, self.y], "->", reward)
        
        # Actualize values in the cell [x, y]
        self.board[self.x, self.y] = 0
        if self.malus[self.x, self.y] == 0:
            self.malus[self.x, self.y] = self.param_malus
        else:
            # Increase the value of malus depending on the number of time we went to the cell
            self.malus[self.x, self.y] += round(self.param_malus / 10, 2)
            
        game_over = self.t > self.max_time
        
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1),
                                self.malus.reshape(self.grid_size, self.grid_size,1)),
                                axis=2)
        
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus
        
        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        
        self.board[self.x,self.y] = 0
        
        self.t = 0
        
        self.malus = np.zeros((self.grid_size, self.grid_size))
        self.malus[self.x, self.y] = self.param_malus
        
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1),
                                self.malus.reshape(self.grid_size, self.grid_size,1)),
                                axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [58]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')

Epoch 000/051 | Loss 0.0064 | Win/lose count 10.5/24.65 (-14.149999999999999) | Epsilon 1.0
Epoch 001/051 | Loss 0.0133 | Win/lose count 12.5/29.830000000000005 (-17.330000000000005) | Epsilon 0.98
Epoch 002/051 | Loss 0.0148 | Win/lose count 6.0/34.58000000000001 (-28.580000000000013) | Epsilon 0.96
Epoch 003/051 | Loss 0.0159 | Win/lose count 15.5/28.970000000000038 (-13.470000000000038) | Epsilon 0.95
Epoch 004/051 | Loss 0.0479 | Win/lose count 11.5/27.610000000000014 (-16.110000000000014) | Epsilon 0.93
Epoch 005/051 | Loss 0.0633 | Win/lose count 8.0/32.01000000000001 (-24.010000000000012) | Epsilon 0.91
Epoch 006/051 | Loss 0.1036 | Win/lose count 9.5/30.210000000000026 (-20.710000000000026) | Epsilon 0.89
Epoch 007/051 | Loss 0.0067 | Win/lose count 7.5/31.180000000000003 (-23.680000000000003) | Epsilon 0.87
Epoch 008/051 | Loss 0.0070 | Win/lose count 13.5/22.480000000000008 (-8.980000000000008) | Epsilon 0.86
Epoch 009/051 | Loss 0.0528 | Win/lose count 10.0/30.48000000000002

In [59]:
HTML(display_videos('cnn_train_explore50.mp4'))

In [60]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')

Win/lose count 21.0/19.42000000000001. Average score (1.5799999999999912)
Win/lose count 18.5/13.729999999999984. Average score (3.1750000000000034)
Win/lose count 20.0/21.400000000000006. Average score (1.650000000000001)
Win/lose count 24.0/13.849999999999982. Average score (3.7750000000000052)
Win/lose count 22.5/21.17000000000002. Average score (3.2860000000000005)
Win/lose count 18.5/19.569999999999997. Average score (2.560000000000002)
Win/lose count 22.5/17.889999999999993. Average score (2.852857142857146)
Win/lose count 24.0/13.689999999999989. Average score (3.7850000000000037)
Win/lose count 17.0/16.420000000000005. Average score (3.4288888888888915)
Win/lose count 21.5/14.879999999999994. Average score (3.7480000000000033)
Win/lose count 25.0/13.139999999999985. Average score (4.48545454545455)
Win/lose count 17.0/18.55. Average score (3.982500000000004)
Win/lose count 22.5/23.54999999999999. Average score (3.59538461538462)
Win/lose count 21.5/15.759999999999993. Average s

In [ ]:
HTML(display_videos('cnn_test_explore10.mp4'))

> The results obtained may be lower than the ones obtained before, but it's only due to the malus we remove each time we get in an empty cell that we have already been to. 

> If ever we add all the points removed (concerning the malus) during the game, we would obtain much better results. Indeed we can see that on the videos. The mouse explores all the fields, going after all cheese it can eat.

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***